<a href="https://colab.research.google.com/github/yixuantt/MSBD5001-Spring-2022/blob/main/msbd5001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab database connect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path="/content/drive/MyDrive/MatchDG/msbd5001-spring-2022"
os.chdir(path)
os.listdir(path)

['train.csv',
 'test.csv',
 'sample_submission.csv',
 '.ipynb_checkpoints',
 'submission.csv',
 'result.csv']

# data preparation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import numpy as np
import warnings
import xgboost as xgb

train = pd.read_csv("/content/drive/MyDrive/MatchDG/msbd5001-spring-2022/train.csv")
test=pd.read_csv("/content/drive/MyDrive/MatchDG/msbd5001-spring-2022/test.csv")
train.drop('id',axis = 1,inplace = True)
test.drop('id',axis = 1,inplace = True)
train.head()

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0


In [ ]:
test.head()

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul)
0,2843.0,156.0,1358.52,730.78,637.85,127.06,94.82,1588.62,45,1,3256.0
1,437.0,137.0,509.43,268.05,243.07,390.86,98.24,1002.76,51,1,491.0
2,826.0,82.0,1232.22,493.42,744.08,516.28,320.15,2200.58,32,0,1381.0
3,861.0,50.0,1512.86,925.51,590.07,380.25,25.80,1929.10,50,0,1377.0
4,1160.0,157.0,890.42,403.91,489.53,266.92,87.63,1251.52,43,0,1844.0


In [ ]:
#NAN
train = train.dropna(how='any', axis = 0)
# #train = train.fillna(method='ffill')
# for i in train.columns:
#     if np.all(pd.notnull(train[i])) == False:
#         print(i)
#         train[i].fillna(train[i].mean(), inplace=True)
train.head()

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0


In [ ]:
#Normalized
# test = (test-test.min())/(test.max()-test.min() )

In [ ]:
features = train.loc[:, train.columns != 'label']
# features = (features-features.min())/(features.max()-features.min())
train.loc[:, train.columns != 'label']  = features
X = train.drop('label', axis=1)
y = train['label']

In [ ]:
N=X.shape[0]
N

86

# model training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:

# gbc_parameters = {'n_estimators':[5,10,30,50,100,200], 
#                   'max_depth':[2,3,4,5,10,15,20,30],
#                   'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
#                   'learning_rate':[0.01,0.1,0.001,0.5]}
# gbc_model = GradientBoostingClassifier(loss='exponential')
# gbc = GridSearchCV(gbc_model, gbc_parameters, cv=5,scoring='accuracy',n_jobs=-1)
# gbc.fit(X, y) 
# # Performance evaluation
# print("The best value for the parameter：:", gbc.best_params_)
# print("Best Model Score:", gbc.best_score_)

In [ ]:
# penaltys = ['l1','l2']
# Cs = [0.001, 0.5,0.01, 0.1, 1,10,100,20]
# stan_parameters =dict(penalty = penaltys, C = Cs)

# r_model = LogisticRegression(solver='liblinear')
# gs3 = GridSearchCV(r_model, stan_parameters, cv=5,scoring='accuracy',n_jobs=-1)
# gs3.fit(X, y) 
# # Performance evaluation
# print("The best value for the parameter：:", gs3.best_params_)
# print("Best Model Score:", gs3.best_score_)

In [ ]:
# tuned_parameters ={'num_leaves': [2,4,8,10,16,31,50],
#                   'min_data_in_leaf': [1,2,3,4,5,6,7,8,10],
#                   'learning_rate':[0.01,0.5,0.1,0.001],
#                   'n_estimators':[5,10,30,50,100,200], 
#                    }

# regress_model = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', metric='auc')
# gs = GridSearchCV(regress_model, tuned_parameters, cv=5,scoring='accuracy',n_jobs=-1)
# gs.fit(X, y) 
# # Performance evaluation
# print("The best value for the parameter：:", gs.best_params_)
# print("Best Model Score:", gs.best_score_)

In [ ]:
from sklearn.model_selection import LeaveOneOut
cv_params = [{'n_estimators': [5,10,30,50,100], 'max_depth': [3,4,5,10,20]}]
model = GridSearchCV( RandomForestClassifier(), cv_params, cv=LeaveOneOut(),scoring='accuracy',n_jobs=-1)
model.fit(X, y) 
# Performance evaluation
print("The best value for the parameter：:", model.best_params_)
print("Best Model Score:", model.best_score_)


The best value for the parameter：: {'max_depth': 4, 'n_estimators': 50}
Best Model Score: 0.9302325581395349


In [ ]:
res=model.predict(test)
res

In [ ]:
TestDataset=pd.read_csv("/content/drive/MyDrive/MatchDG/msbd5001-spring-2022/test.csv")
test_ids=TestDataset['id']
submission = pd.DataFrame(res,columns=['label'])
submission.insert(0, 'id', test_ids)
submission['label']=submission['label'].astype(int)
submission

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [ ]:
submission.to_csv('result.csv', index = False)